In [13]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from helper_functions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import pandas as pd
import numpy as np
import json
import os
import sys
import sklearn

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFE

src_dir = os.path.join(os.getcwd(), '..\\..', 'src')
sys.path.append(src_dir)

In [23]:
data = pd.read_csv('../../data/01_raw/Absenteeism_at_work.csv', delimiter=';')
print(data.shape)
data.head()

(740, 21)


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [24]:
list(data.columns)

['ID',
 'Reason for absence',
 'Month of absence',
 'Day of the week',
 'Seasons',
 'Transportation expense',
 'Distance from Residence to Work',
 'Service time',
 'Age',
 'Work load Average/day ',
 'Hit target',
 'Disciplinary failure',
 'Education',
 'Son',
 'Social drinker',
 'Social smoker',
 'Pet',
 'Weight',
 'Height',
 'Body mass index',
 'Absenteeism time in hours']

In [25]:
data.describe()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,...,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,18.017568,19.216216,6.324324,3.914865,2.544595,221.329730,29.631081,12.554054,36.450000,271.490235,...,0.054054,1.291892,1.018919,0.567568,0.072973,0.745946,79.035135,172.114865,26.677027,6.924324
std,11.021247,8.433406,3.436287,1.421675,1.111831,66.952223,14.836788,4.384873,6.478772,39.058116,...,0.226277,0.673238,1.098489,0.495749,0.260268,1.318258,12.883211,6.034995,4.285452,13.330998
min,1.000000,0.000000,0.000000,2.000000,1.000000,118.000000,5.000000,1.000000,27.000000,205.917000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,56.000000,163.000000,19.000000,0.000000
25%,9.000000,13.000000,3.000000,3.000000,2.000000,179.000000,16.000000,9.000000,31.000000,244.387000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,69.000000,169.000000,24.000000,2.000000
50%,18.000000,23.000000,6.000000,4.000000,3.000000,225.000000,26.000000,13.000000,37.000000,264.249000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,83.000000,170.000000,25.000000,3.000000
75%,28.000000,26.000000,9.000000,5.000000,4.000000,260.000000,50.000000,16.000000,40.000000,294.217000,...,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,89.000000,172.000000,31.000000,8.000000
max,36.000000,28.000000,12.000000,6.000000,4.000000,388.000000,52.000000,29.000000,58.000000,378.884000,...,1.000000,4.000000,4.000000,1.000000,1.000000,8.000000,108.000000,196.000000,38.000000,120.000000


Columns to use as predictors:
- reason -> make into dummy variables
- age
- day of the week -> make into dummy variables for each
- body mass index
- son
- distance to work


Targets for logistic regression?

- Disciplinary
- Hit target (turn into dummy)

In [32]:
data['medical_consult_dummy'] = (data['Reason for absence'] == 23).astype(int)
data['dental_consult_dummy'] = (data['Reason for absence'] == 28).astype(int)
data['genitourinary_dummy'] = (data['Reason for absence'] == 14).astype(int)
data.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours,medical_consult_dummy,dental_consult_dummy,genitourinary_dummy
0,11,26,7,3,1,289,36,13,33,239.554,...,1,0,1,90,172,30,4,0,0,0
1,36,0,7,3,1,118,13,18,50,239.554,...,1,0,0,98,178,31,0,0,0,0
2,3,23,7,4,1,179,51,18,38,239.554,...,1,0,0,89,170,31,2,1,0,0
3,7,7,7,5,1,279,5,14,39,239.554,...,1,1,0,68,168,24,4,0,0,0
4,11,23,7,5,1,289,36,13,33,239.554,...,1,0,1,90,172,30,2,1,0,0


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 29 columns):
ID                                 740 non-null int64
Reason_for_absence                 740 non-null int64
Month_of_absence                   740 non-null int64
Day_of_the_week                    740 non-null int64
Seasons                            740 non-null int64
Transportation_expense             740 non-null int64
Distance_from_Residence_to_Work    740 non-null int64
Service_time                       740 non-null int64
Age                                740 non-null int64
Work_load_Average/day_             740 non-null float64
Hit_target                         740 non-null int64
Disciplinary_failure               740 non-null int64
Education                          740 non-null int64
Son                                740 non-null int64
Social_drinker                     740 non-null int64
Social_smoker                      740 non-null int64
Pet                          

In [36]:
dow = pd.get_dummies(data['Day of the week'])
dow.columns = ['mon_dummy', 'tue_dummy', 'wed_dummy', 'thu_dummy' ,'fri_dummy']
data = pd.concat([data, dow], axis=1)
print(data.shape)

(740, 29)


In [42]:
data.columns = data.columns.str.replace(' ', '_')

#### Start modeling

In [44]:
data.columns

Index(['ID', 'Reason_for_absence', 'Month_of_absence', 'Day_of_the_week',
       'Seasons', 'Transportation_expense', 'Distance_from_Residence_to_Work',
       'Service_time', 'Age', 'Work_load_Average/day_', 'Hit_target',
       'Disciplinary_failure', 'Education', 'Son', 'Social_drinker',
       'Social_smoker', 'Pet', 'Weight', 'Height', 'Body_mass_index',
       'Absenteeism_time_in_hours', 'medical_consult_dummy',
       'dental_consult_dummy', 'genitourinary_dummy', 'mon_dummy', 'tue_dummy',
       'wed_dummy', 'thu_dummy', 'fri_dummy'],
      dtype='object')

In [52]:
feature_cols = ['Age', 'Body_mass_index', 'Son', 'Distance_from_Residence_to_Work',
                'mon_dummy', 'wed_dummy', 'thu_dummy', 'fri_dummy', 'medical_consult_dummy',
                'dental_consult_dummy', 'genitourinary_dummy']
input_data = data[feature_cols]
input_data = sm.add_constant(input_data)
target = data.Absenteeism_time_in_hours

In [53]:
input_data.shape

(740, 12)

#### Statsmodels

In [54]:
model = sm.OLS(target, input_data)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                               
=====================================================================================
Dep. Variable:     Absenteeism_time_in_hours   R-squared:                       0.091
Model:                                   OLS   Adj. R-squared:                  0.078
Method:                        Least Squares   F-statistic:                     6.659
Date:                       Mon, 29 Jul 2019   Prob (F-statistic):           1.11e-10
Time:                               11:35:05   Log-Likelihood:                -2930.7
No. Observations:                        740   AIC:                             5885.
Df Residuals:                            728   BIC:                             5941.
Df Model:                                 11                                         
Covariance Type:                   nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const                              10.3639      3.612      2.869      0.004       3.273      17.455
Age                                 0.1969      0.087      2.258      0.024       0.026       0.368
Body_mass_index                    -0.2707      0.132     -2.050      0.041      -0.530      -0.011
Son                                 1.3352      0.444      3.005      0.003       0.463       2.208
Distance_from_Residence_to_Work    -0.0413      0.034     -1.215      0.225      -0.108       0.025
mon_dummy                           0.6572      1.456      0.451      0.652      -2.200       3.515
wed_dummy                          -1.3574      1.470     -0.923      0.356      -4.244       1.529
thu_dummy                          -3.6571      1.550     -2.360      0.019      -6.699      -0.615
fri_dummy                          -3.2795      1.525     -2.150      0.032      -6.274      -0.285
medical_consult_dummy              -5.8977      1.217     -4.847      0.000      -8.286      -3.509
dental_consult_dummy               -6.1034      1.381     -4.418      0.000      -8.816      -3.391
genitourinary_dummy                -0.8273      3.039     -0.272      0.786      -6.793       5.138
==============================================================================
Omnibus:                      812.850   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            43497.168
Skew:                           5.349   Prob(JB):                         0.00
Kurtosis:                      39.004   Cond. No.                         430.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Sklearn

In [70]:
X_train, X_test, y_train, y_test = train_test_split(input_data, target, random_state = 101)

In [71]:
print(X_train.shape)

(555, 12)


In [72]:
print(y_train.shape)

(555,)


In [73]:
print(X_test.shape)

(185, 12)


In [74]:
print(y_test.shape)

(185,)


In [75]:
linreg = LinearRegression()
model1 = linreg.fit(input_data, target)

In [76]:
model1.coef_

array([ 0.        ,  0.19689475, -0.27068174,  1.33523026, -0.0413182 ,
        0.65715515, -1.35736713, -3.6570804 , -3.27946223, -5.89771016,
       -6.10344816, -0.82731321])

In [80]:
linreg = LinearRegression()
model1 = linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

In [84]:
np.sqrt(mean_squared_error(y_test, y_pred))

11.466648713627409

In [3]:
absent = pd.read_csv('Absenteeism_at_work.csv', delimiter=';')
print(absent.shape)
absent.head()

(740, 21)


,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [6]:
absent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
ID                                 740 non-null int64
Reason for absence                 740 non-null int64
Month of absence                   740 non-null int64
Day of the week                    740 non-null int64
Seasons                            740 non-null int64
Transportation expense             740 non-null int64
Distance from Residence to Work    740 non-null int64
Service time                       740 non-null int64
Age                                740 non-null int64
Work load Average/day              740 non-null float64
Hit target                         740 non-null int64
Disciplinary failure               740 non-null int64
Education                          740 non-null int64
Son                                740 non-null int64
Social drinker                     740 non-null int64
Social smoker                      740 non-null int64
Pet                          

In [5]:
absent.describe()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,...,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,18.017568,19.216216,6.324324,3.914865,2.544595,221.329730,29.631081,12.554054,36.450000,271.490235,...,0.054054,1.291892,1.018919,0.567568,0.072973,0.745946,79.035135,172.114865,26.677027,6.924324
std,11.021247,8.433406,3.436287,1.421675,1.111831,66.952223,14.836788,4.384873,6.478772,39.058116,...,0.226277,0.673238,1.098489,0.495749,0.260268,1.318258,12.883211,6.034995,4.285452,13.330998
min,1.000000,0.000000,0.000000,2.000000,1.000000,118.000000,5.000000,1.000000,27.000000,205.917000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,56.000000,163.000000,19.000000,0.000000
25%,9.000000,13.000000,3.000000,3.000000,2.000000,179.000000,16.000000,9.000000,31.000000,244.387000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,69.000000,169.000000,24.000000,2.000000
50%,18.000000,23.000000,6.000000,4.000000,3.000000,225.000000,26.000000,13.000000,37.000000,264.249000,...,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,83.000000,170.000000,25.000000,3.000000
75%,28.000000,26.000000,9.000000,5.000000,4.000000,260.000000,50.000000,16.000000,40.000000,294.217000,...,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,89.000000,172.000000,31.000000,8.000000
max,36.000000,28.000000,12.000000,6.000000,4.000000,388.000000,52.000000,29.000000,58.000000,378.884000,...,1.000000,4.000000,4.000000,1.000000,1.000000,8.000000,108.000000,196.000000,38.000000,120.000000


In [38]:
absent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 21 columns):
ID                                 740 non-null int64
Reason for absence                 740 non-null int64
Month of absence                   740 non-null int64
Day of the week                    740 non-null int64
Seasons                            740 non-null int64
Transportation expense             740 non-null int64
Distance from Residence to Work    740 non-null int64
Service time                       740 non-null int64
Age                                740 non-null int64
Work load Average/day              740 non-null float64
Hit target                         740 non-null int64
Disciplinary failure               740 non-null int64
Education                          740 non-null int64
Son                                740 non-null int64
Social drinker                     740 non-null int64
Social smoker                      740 non-null int64
Pet                          

In [7]:
ads = pd.read_csv('../advertising/advertising.csv')

In [9]:
print(ads.shape)
ads.head()

(1000, 10)


,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0


In [10]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
Daily Time Spent on Site    1000 non-null float64
Age                         1000 non-null int64
Area Income                 1000 non-null float64
Daily Internet Usage        1000 non-null float64
Ad Topic Line               1000 non-null object
City                        1000 non-null object
Male                        1000 non-null int64
Country                     1000 non-null object
Timestamp                   1000 non-null object
Clicked on Ad               1000 non-null int64
dtypes: float64(3), int64(3), object(4)
memory usage: 62.5+ KB


In [25]:
virus = pd.read_csv('../virusshare/2010-11.txt', header = None)
len(virus.iloc[0].values[0])#.split(' ')

334

In [30]:
# with ('../virusshare/features/features.json', 'rb') as file:
#     feat = json.load()
    
with open('../virusshare/features/features.json') as json_file:
    data = json.load(json_file)

In [34]:
data.keys()

dict_keys(['call_feature', 'registry_feature', 'file_feature', 'misc_feature'])

In [37]:
data['call_feature'].keys()

dict_keys(['function', 'call_file'])

In [28]:
pd.read_json('../virusshare/features/features.json')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.